In [209]:
plurals = {"ẓarf": "ẓuru:f",
          "ʔana:": "ʔiḥna:",
           "ʔinta": "ʔintu:",
           "ʔinti:": "ʔintu:",
           "huwwa": "humma",
           "hiyya": "humma"}

pronouns = {"1": ["ʔana:", "ʔiḥna:"],
       "2": ["ʔinta", "ʔinti:", "ʔintu:"],
       "3": ["huwwa", "hiyya", "humma"]}

        # lemma: [1/2-perfect stem, present stem]
verbs = {"fataḥ": ["fataḥ", "ftaḥ"],
         "ša:f": ["šuf", "šu:f"]}

prepositions = {"INSTRUMENT": "bi-",
                "INDIRECT": "li-",
                "CONCOMITATIVE": "maʕa"}

perfect_inflection = {"ʔana:": ["-t"],
                      "ʔiḥna:": ["-na:"],
                      "ʔinta": ["-t"],
                      "ʔinti:": ["-ti:"],
                      "ʔintu:": ["-tu:"],
                      "huwwa": [],
                      "hiyya": ["-it"],
                      "humma": ["-u:"]}

In [94]:
def pluralize(w):
    if w in pronouns:
        return pronouns[w]
    else:
        return nouns[w]

In [201]:
def get_pronoun(id):
    s = semantics[id]
    
    person = s[1]
    gender = s[2]
    number = s[3]
    animacy = s[5]

    if (gender == "FEMININE" and person != "1") or (number == "PLURAL" and animacy == "INANIMATE" and gender not in "12"):
        return pronouns[person][-2]
    elif number == "PLURAL":
        return pronouns[person][-1]
    else:
        return pronouns[person][0]

In [185]:
def conjugate_verb(id_verb, id_subject):
    """ returns list of strings corresponding to conjugated verb (not including obj. suffixes) """
    
    s_verb = semantics[id_verb]
    s_subject = semantics[id_subject]
    
    root = s_verb[0]
    tense = s_verb[1]
    
    person = s_subject[1]
    gender = s_subject[2]
    number = s_subject[3]
    
    animacy = s_subject[5]

    if animacy == "INANIMATE" and number == "PLURAL" and person not in "12":
        gender = "FEMININE"
        number = "SINGULAR"
        
    subject_pronoun = get_pronoun(id_subject)
    
    verb_morphology = []
    
    if tense == "PERFECT":
        verb_morphology = perfect_inflection[subject_pronoun]
        if person in "12":
            return [verbs[root][0]] + verb_morphology
        else:
            return [root] + verb_morphology
    
    # no modals/aux verbs yet
    
    return None
    

In [186]:
def print_sentence(ls):
    """ Prints a list of sentence elements, joined with spaces """
    new_list = []
    for element in ls:
        if element[0]=='-' and new_list[-1]==' ':
            new_list = new_list[:-1]
        new_list.append(element)
        if element[-1]=='-':
            continue
        new_list.append(" ")
    sentence = "".join(new_list[:-1])
    return sentence + "."

In [187]:
def CP():
    sentence = DP("SUBJECT") + VP("VERB")
    
    args = list(semantics.keys())
    for element in ["SUBJECT","VERB","OBJECT","INDIRECT","NEGATIVE","NULL"]:
        if element in args:
            args.remove(element)
    
    for argument in args:
        sentence = sentence + PP(argument)
    return sentence

In [188]:
def DP(id="NULL"):
    s = semantics[id]
    # if argument is bare determiner (pronoun, etc.) ...
    if s[0]=="":
        # retrieve pronoun for argument vector
        pro = get_pronoun(id)
        return [pro]
    else:
        return D(id) + NP(id)

In [207]:
def D(id="NULL"):
    s = semantics[id]
    if "DEFINITE" in s:
        return ["ʔil-"]
    else:
        return []

In [190]:
def NP(id="NULL"):
    s = semantics[id]
    w = s[0]
    if "PLURAL" in s:
        w = pluralize(w)
    return [w]

In [191]:
def VP(id="VERB"):
    return V(id) + DP("OBJECT")

In [192]:
def PP(id="NULL"):
    s = semantics[id]
    w = prepositions[id]
    return [w] + DP(id)

In [193]:
def V(id="VERB"):
    vb = conjugate_verb("VERB", "SUBJECT")
    
    # attach direct and indirect object suffixes here
    
    if semantics["NEGATIVE"]==False:
        return vb
    else:
        return ["ma:-"] + vb + ["-š"]

In [244]:
semantics = {# NOUN: [lemma, person, gender, number, definiteness, animacy]
             "SUBJECT": ["","1","FEMININE","PLURAL","",""],
             
             "OBJECT": ["ẓarf","","MASCULINE","SINGULAR","DEFINITE","INANIMATE"],

             "INSTRUMENT": ["sikki:naḧ","","FEMININE","SINGULAR","INDEFINITE","INANIMATE"],

             "NEGATIVE": True,
             
             # VERB: [root, TAM, modal]
             "VERB": ["fataḥ", "PERFECT", ""],
    
             "NULL": []}

In [245]:
s = CP()
print(s)
print_sentence(s)

['ʔiḥna:', 'ma:-', 'fataḥ', '-na:', '-š', 'ʔil-', 'ẓarf', 'bi-', 'sikki:naḧ']


'ʔiḥna: ma:-fataḥ-na:-š ʔil-ẓarf bi-sikki:naḧ.'

In [248]:
def transliterate_to_ar(s):
    
    s = "$" + s
    
    en_cons = "ʔbtgḥxdrzsšṣḍṭẓʕġfqklmnhwyḧ "
    ar_cons = "ابتجحخدرزسشصضطظعغفقكلمنهوية "
    
    new_s = ""
    
    for n in range(1, len(s)):
        if s[n]==s[n-1]:
            new_s += "ّ"

        
        elif s[n] in en_cons:
            i = en_cons.find(s[n])
            new_s += ar_cons[i]
            
        elif s[n] == ":":
            if s[n-1] == "a":
                new_s += "ا"
            elif s[n-1] in "ei":
                new_s += "ي"
            elif s[n-1] in "ou":
                new_s += "و"
            
    return new_s

In [249]:
transliterate_to_ar(print_sentence(s))

'احنا مافتحناش الظرف بسكّينة'